# Importing Libraries

In [1]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

from fosforio import snowflake

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


# Fetching Data

In [2]:
# To read a specific dataset published from a snowflake connection
df = snowflake.get_dataframe("MASTER")
df.head()

,ID,GENDER,DOB,CITY,STATE,AGE,MONTHLY_INCOME,EMPLOYER_NAME,SALARY_ACCOUNT,LOAN_AMOUNT_APPLIED,...,QUARTER,MOBILE_VERIFIED,FILLED_FORM,DEVICE_TYPE,VAR2,SOURCE,VAR4,VAR5,VAR1,DISBURSED
0,ID000977Z13,Male,10/1/1988,New York,New York,36,1313.242865,Hcl Global Systems,Bank of Hope,900000.0,...,4,Y,Y,Mobile,E,S127,7.0,14,HBXX,1.0
1,ID000626U25,Male,4/1/1992,New York,New York,32,1270.880192,Synechron,Bank of Marin,900000.0,...,2,Y,N,Web-browser,G,S151,5.0,10,HBXB,1.0
2,ID019473Z30,Male,02/1/1996,New York,New York,28,500.000000,Syntel,Exchange Bank,10000.0,...,2,Y,N,Web-browser,B,S144,4.0,11,HBXC,0.0
3,ID000062G50,Male,3/6/1989,New York,New York,35,1497.000000,Uber Technologies,Bank of Marin,900000.0,...,1,Y,Y,Web-browser,B,S133,4.0,0,HAXB,1.0
4,ID000935I02,Female,30/8/1996,New York,New York,28,903.000000,Hcl America,CIT Bank,700000.0,...,4,N,Y,Web-browser,B,S160,1.0,3,HBXX,0.0


# Preprocessing

In [3]:
data = df.drop(["ID", "LEAD_CREATION_DATE", "DEVICE_TYPE", "SALARY_ACCOUNT", "CITY", "DOB", "EMPLOYER_NAME","YEAR","QUARTER","MONTH"], axis=1)

In [4]:
data.head()

,GENDER,STATE,AGE,MONTHLY_INCOME,LOAN_AMOUNT_APPLIED,LOAN_TENURE_APPLIED,EXISTING_EMI,MOBILE_VERIFIED,FILLED_FORM,VAR2,SOURCE,VAR4,VAR5,VAR1,DISBURSED
0,Male,New York,36,1313.242865,900000.0,3.0,139.0,Y,Y,E,S127,7.0,14,HBXX,1.0
1,Male,New York,32,1270.880192,900000.0,3.0,0.0,Y,N,G,S151,5.0,10,HBXB,1.0
2,Male,New York,28,500.000000,10000.0,0.0,100.0,Y,N,B,S144,4.0,11,HBXC,0.0
3,Male,New York,35,1497.000000,900000.0,3.0,140.0,Y,Y,B,S133,4.0,0,HAXB,1.0
4,Female,New York,28,903.000000,700000.0,3.0,0.0,N,Y,B,S160,1.0,3,HBXX,0.0


In [5]:
data = data.dropna()

In [6]:
cat_col = []
num_col = []
target = "DISBURSED"

for k, v in dict(data.dtypes).items():
    if k != target:
        if v == "O":
            cat_col.append(k)
        else:
            num_col.append(k)

# Train Test Split

In [7]:
used_cols = [c for c in data.columns.tolist() if c not in [target]]
X = data[used_cols]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building and Training Model Pipeline

In [8]:
# making pipeline
scaler = RobustScaler()
encoder = OneHotEncoder(handle_unknown="ignore")
# putting numeric columns to scaler and categorical to encoder
num_transformer = make_pipeline(scaler)
cat_transformer = make_pipeline(encoder)

# getting together our scaler and encoder with preprocessor
preprocessor = ColumnTransformer(
      transformers=[('num', num_transformer, num_col),
                    ('cat', cat_transformer, cat_col)])

In [9]:
# choosing model
model_name = RandomForestClassifier(n_estimators = 500, max_depth=8, random_state=25, max_samples=0.6)

# giving all values to pipeline
pipe = make_pipeline(preprocessor,model_name)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('robustscaler',
                                                                   RobustScaler())]),
                                                  ['AGE', 'MONTHLY_INCOME',
                                                   'LOAN_AMOUNT_APPLIED',
                                                   'LOAN_TENURE_APPLIED',
                                                   'EXISTING_EMI', 'VAR4',
                                                   'VAR5']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['GENDER', 'STATE',
                                                   'MOBILE_VERIFIED',
                                                   'FILLED_FORM', 'VAR2',
                                                   'SOURCE', 'VAR1'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=8, max_samples=0.6,
                                        n_estimators=500, random_state=25))])

# Prediction

In [10]:
# make predictions on test set
y_pred = pipe.predict(X_test)


In [13]:
pred = pipe.predict(data)

In [15]:
pd.DataFrame({"prediction":pred}).to_dict()

{'prediction': {0: 1.0,
  1: 1.0,
  2: 0.0,
  3: 1.0,
  4: 0.0,
  5: 0.0,
  6: 1.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 1.0,
  13: 1.0,
  14: 1.0,
  15: 1.0,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.0,
  20: 0.0,
  21: 0.0,
  22: 0.0,
  23: 1.0,
  24: 1.0,
  25: 1.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: 0.0,
  35: 0.0,
  36: 1.0,
  37: 1.0,
  38: 0.0,
  39: 0.0,
  40: 0.0,
  41: 0.0,
  42: 1.0,
  43: 0.0,
  44: 0.0,
  45: 1.0,
  46: 1.0,
  47: 0.0,
  48: 0.0,
  49: 1.0,
  50: 1.0,
  51: 1.0,
  52: 1.0,
  53: 1.0,
  54: 0.0,
  55: 0.0,
  56: 0.0,
  57: 0.0,
  58: 0.0,
  59: 1.0,
  60: 0.0,
  61: 0.0,
  62: 1.0,
  63: 0.0,
  64: 1.0,
  65: 1.0,
  66: 0.0,
  67: 0.0,
  68: 0.0,
  69: 1.0,
  70: 0.0,
  71: 1.0,
  72: 0.0,
  73: 1.0,
  74: 0.0,
  75: 0.0,
  76: 0.0,
  77: 0.0,
  78: 1.0,
  79: 0.0,
  80: 0.0,
  81: 1.0,
  82: 0.0,
  83: 1.0,
  84: 1.0,
  85: 1.0,
  86: 0.0,
  87: 0.0,
  88: 1.0,
  89: 1.0,
  90: 

# Model Registrartion

In [15]:
# !pip install fosforml

In [11]:
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [40]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame.from_dict(payload_dict)
#     prediction = pd.DataFrame({"prediction":data})
    return data

In [41]:
payload = X_test.head(100).to_dict()
req = requests.Request()
req.json = {"payload":payload}
y = req
yo = score(pipe, y)
yo

,GENDER,STATE,AGE,MONTHLY_INCOME,LOAN_AMOUNT_APPLIED,LOAN_TENURE_APPLIED,EXISTING_EMI,MOBILE_VERIFIED,FILLED_FORM,VAR2,SOURCE,VAR4,VAR5,VAR1
118356,Male,Texas,45,1172.000000,800000.0,2.0,0.0,Y,Y,E,S122,2.0,0,HBXX
222522,Male,California,44,1257.000000,600000.0,3.0,127.0,Y,Y,E,S159,7.0,17,HAXB
202894,Male,Texas,45,1186.350732,900000.0,3.0,130.0,Y,Y,E,S151,4.0,10,HBXX
321808,Male,Massachusetts,32,978.846220,800000.0,3.0,0.0,Y,Y,E,S133,5.0,0,HAXA
17625,Female,New York,41,791.022174,700000.0,3.0,169.0,Y,Y,B,S157,5.0,17,HBXD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72150,Female,Pennsylvania,41,1746.000000,500000.0,3.0,0.0,Y,Y,F,S157,5.0,12,HBXX
96691,Male,California,43,1675.578246,800000.0,3.0,0.0,Y,Y,F,S151,3.0,12,HAXM
329847,Male,Texas,38,1123.911220,900000.0,3.0,0.0,Y,Y,E,S161,7.0,16,HBXX
336095,Male,Texas,40,1000.359324,700000.0,3.0,0.0,N,N,B,S134,1.0,14,HBXX


In [32]:
req.json

{'payload': {'GENDER': {118356: 'Male',
   222522: 'Male',
   202894: 'Male',
   321808: 'Male',
   17625: 'Female',
   346017: 'Male',
   244880: 'Male',
   255258: 'Male',
   194217: 'Female',
   348503: 'Male',
   64571: 'Female',
   239599: 'Male',
   177933: 'Male',
   144557: 'Female',
   88950: 'Male',
   183090: 'Female',
   50412: 'Male',
   39125: 'Male',
   17198: 'Female',
   260030: 'Male',
   359907: 'Male',
   196424: 'Female',
   284994: 'Female',
   289837: 'Male',
   273320: 'Male',
   317349: 'Female',
   269281: 'Female',
   65852: 'Female',
   341839: 'Male',
   315337: 'Male',
   171273: 'Female',
   265616: 'Female',
   187144: 'Female',
   196694: 'Male',
   281478: 'Male',
   203758: 'Male',
   145418: 'Male',
   291635: 'Male',
   341721: 'Male',
   168838: 'Male',
   8814: 'Female',
   99501: 'Male',
   273310: 'Male',
   23962: 'Male',
   228765: 'Male',
   261306: 'Male',
   50482: 'Male',
   263420: 'Female',
   305: 'Male',
   297647: 'Female',
   188829:

In [ ]:
## registering the model in Fosfor.
model_reg = register_model(clf,
               score, 
               name="Customer_Segmentation_Dtree_Cluster_Classifier", 
               description="Customer_Segmentation_Dtree_Cluster_Classifier",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=clf_prediction,
               prob=clf_prediction,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)